# Défi Grosses Data 2018 - Réseau de neurones avec scikit-learn

In [10]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import imp
import Annex
imp.reload(Annex)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [2]:
X_train,X_test,Y_train,Y_test,X,Y,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.3,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='week_circle')#'drop'

We are addressing your request.
Data has been imported. Size: (189280, 31)
Months converted to int.
Dummies added.
Date transformed in a projection of the week number on a circle.
26528 data points deleted. 14.02 %
Train size: 113926, Test size: 48826
Data scaled


In [14]:
X.dtypes

capeinsSOL0        float64
ciwcH20            float64
clwcH20            float64
ffH10              float64
flir1SOL0          float64
fllat1SOL0         float64
flsen1SOL0         float64
flvis1SOL0         float64
hcoulimSOL0        float64
huH2               float64
iwcSOL0            float64
nbSOL0_HMoy        float64
nH20               float64
ntSOL0_HMoy        float64
pMER0              float64
rr1SOL0            float64
rrH20              float64
tH2                float64
tH2_VGrad_2.100    float64
tH2_XGrad          float64
tH2_YGrad          float64
tpwHPA850          float64
ux1H10             float64
vapcSOL0           float64
vx1H10             float64
ech                  int64
mois                 int32
insee_6088001        uint8
insee_31069001       uint8
insee_33281001       uint8
insee_35281001       uint8
insee_59343001       uint8
insee_67124001       uint8
insee_75114001       uint8
ddH10_rose4_1.0      uint8
ddH10_rose4_2.0      uint8
ddH10_rose4_3.0      uint8
d

In [3]:
#mlp=MLPRegressor(hidden_layer_sizes =(72,))
mlp=XGBRegressor(max_depth=7,n_estimators=150)
nnetOpt=mlp.fit(X_train, Y_train)

In [4]:
# prévision de l'échantillon test
Y_pred = nnetOpt.predict(X_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))

MSE = 1.0648135312
R2 = 0.97287704007


In [6]:
Annex.generate_submission_file('submission_XGB_19Dec2017_15h30.csv', 
                         nnetOpt, 
                         scaler, 
                         add_dummies=True, 
                         var_dummies=['insee','ddH10_rose4'], 
                         convert_month2int=True, 
                         date_method='week_number', 
                         fillna_method='imputer' )

Months converted to int.
Dummies added.
Date transformed in week number.


'File ./../data_meteo/submission_XGB_19Dec2017_15h30.csv generated.'

In [8]:
Annex.combine_submission_files('submission_combinaison_2NN_2XGB_19Dec2017_15h30.csv', 
                               names=['submission_NN80_17Dec2017_21h.csv',
                                      'submission_XGB_maxdepth6_150estimators_18Dec2017_7h.csv',
                                      'submission_NN70_18Dec2017_9h45.csv',
                                      #'submission_NN71_18Dec2017_10h45.csv',
                                      #'submission_NN75_18Dec2017_10h45.csv',
                                      #'submission_NN72_18Dec2017_10h45.csv',
                                      'submission_XGB_19Dec2017_15h30.csv',
                                     ])

(21168,)
(21168,)
(21168,)
(21168,)
(21168, 4)


'File ./../data_meteo/submission_combinaison_2NN_2XGB_19Dec2017_15h30.csv generated.'

### Sammy, optimize!

In [4]:
import sys
sys.path.append("./../")
import Sammy
import pickle
import os
path='./results/Resultats.pickle'
if os.path.exists(path):
    Resultats=Sammy.Results()#Resultats=pickle.load(open(path, 'rb'))
else:
    Resultats=Sammy.Results()
Resultats.say_hello()

Welcome, I am Sammy Nimizz.
May I help you?


In [11]:
Resultats.random_start(X_train,Y_train,nb_run=10,methods=XGBRegressor)
f=open('./results/Resultats.pickle', 'wb')
pickle.dump(Resultats,f)
f.close()

Run 1/10
Running XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 150, 'min_child_weight': 1, 'max_depth': 13, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
Score: 9.766e-01

Run 2/10
Running XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 250, 'min_child_weight': 1, 'max_depth': 12, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
Score: 9.770e-01

Run 3/10
Running XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread'

In [ ]:
# prévision de l'échantillon test
Y_pred = nnetOpt.predict(X_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))

In [15]:
print(Sammy.Results(Resultats.bests(nb=10)))

10 Results:
XGBRegressor:
Parameters:
{'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 250, 'min_child_weight': 1, 'max_depth': 12, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
CV Score:   9.770e-01

XGBRegressor:
Parameters:
{'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 14, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
CV Score:   9.767e-01

XGBRegressor:
Parameters:
{'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 150,

In [16]:
X_train.shape

(113926, 39)

In [14]:
Resultats.test(X_train, Y_train, X_test, Y_test, nb_test=5)

Testing XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 250, 'min_child_weight': 1, 'max_depth': 12, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
Test Score= 0.863882555807
Testing XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 14, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
Test Score= 0.880543340992
Testing XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, '

In [17]:
param={'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 250, 'min_child_weight': 1, 'max_depth': 12, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
method=XGBRegressor(**param)
predictor=method.fit(X_train, Y_train)

In [19]:
Y_pred = predictor.predict(X_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))

MSE = 0.863882555807
R2 = 0.977995159473


In [20]:
Annex.generate_submission_file('submission_XGB_13Dec_20h.csv', 
                         predictor, 
                         scaler, 
                         add_dummies=True, 
                         var_dummies=['insee','ddH10_rose4'], 
                         convert_month2int=True, 
                         date_method='week_number', 
                         fillna_method='zeros' )

Months converted to int.
Dummies added.
Date transformed in week number.


'File ./../data_meteo/submission_XGB_13Dec_20h.csv generated.'

# Super Learner

In [5]:
mlp1=MLPRegressor(hidden_layer_sizes =(70,))
mlp2=MLPRegressor(hidden_layer_sizes =(80,))
xgb1=XGBRegressor(max_depth=6,n_estimators=150)
xgb2=XGBRegressor(max_depth=7,n_estimators=150)
nnet1=mlp1.fit(X_train, Y_train)
nnet2=mlp2.fit(X_train, Y_train)
xgboost1=xgb1.fit(X_train, Y_train)
xgboost2=xgb2.fit(X_train, Y_train)

In [12]:
xgb3=XGBRegressor(max_depth=7,n_estimators=150)
xgboost3=xgb3.fit(X_train, Y_train)

In [6]:
ypred1=nnet1.predict(X_test)
ypred2=nnet2.predict(X_test)
ypred3=xgboost1.predict(X_test)
ypred4=xgboost2.predict(X_test)
L2=len(ypred1)
X_super_test=np.concatenate([ypred1.reshape(L2,1),ypred2.reshape(L2,1),ypred3.reshape(L2,1),ypred4.reshape(L2,1)],axis=1)
Y_PRED=X_super_test.mean(axis=0)
Y_PRED.shape

(4,)

AttributeError: 'numpy.ndarray' object has no attribute 'dtypes'

In [10]:
ypredtrain1=nnet1.predict(X_train)
ypredtrain2=nnet2.predict(X_train)
ypredtrain3=xgboost1.predict(X_train)
L=len(ypredtrain1)
X_super_train=np.concatenate([ypredtrain1.reshape(L,1),ypredtrain2.reshape(L,1),ypredtrain3.reshape(L,1)],axis=1)






In [24]:
#SuperLearner=LinearRegression()
SuperLearner=MLPRegressor(hidden_layer_sizes=(50,))
SuperLearner=SuperLearner.fit(X_super_train, Y_train)

In [25]:
Y_pred = SuperLearner.predict(X_super_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))

MSE = 1.0993138716
R2 = 0.971998246438


In [15]:
Annex.generate_submission_file('submission_weekC_2NN_3XGB_19Dec2017_19h.csv', 
                         [nnet1,nnet2,xgboost1,xgboost2,xgboost3,None], 
                         scaler, 
                         add_dummies=True, 
                         var_dummies=['insee','ddH10_rose4'], 
                         convert_month2int=True, 
                         date_method='week_circle', 
                         fillna_method='imputer' )

Months converted to int.
Dummies added.
Date transformed in a projection of the week number on a circle.
(21168,)


'File ./../data_meteo/submission_weekC_2NN_3XGB_19Dec2017_19h.csv generated.'